**Preparation**

Run the next 4 code blocks before Experiment #1 and Experiment #2

In [104]:
import urllib.request
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential, model_from_json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [5]:
try:
  urllib.request.urlretrieve ("https://raw.githubusercontent.com/whittlem/colabnotebooks/main/datasets/btc-gbp-3600.csv", "btc-gbp-3600.csv")
  print('Success: downloaded btc-gbp-3600.csv.')
except Exception as e:
  print('Data load error: ',e)
try:
  df = pd.read_csv('./btc-gbp-3600.csv', index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded btc-gbp-3600.csv.
Success: Data loaded into dataframe.


In [6]:
df.shape

(29981, 73)

In [8]:
print (f'{df.shape[0] / 24} days of hourly trading data!')

1249.2083333333333 days of hourly trading data!


**Experiment #1**

* Single input
* Single output

In [9]:
df.head()

,market,granularity,low,high,open,close,volume,close_pc,close_cpc,cma,sma20,sma50,sma200,ema8,ema12,ema26,goldencross,deathcross,fbb_mid,fbb_upper0_236,fbb_upper0_382,fbb_upper0_5,fbb_upper0_618,fbb_upper0_764,fbb_upper1,fbb_lower0_236,fbb_lower0_382,fbb_lower0_5,fbb_lower0_618,fbb_lower0_764,fbb_lower1,rsi14,macd,signal,obv,obv_pc,ema13,elder_ray_bull,elder_ray_bear,eri_buy,eri_sell,ema8gtema12,ema8gtema12co,ema8ltema12,ema8ltema12co,ema12gtema26,ema12gtema26co,ema12ltema26,ema12ltema26co,sma50gtsma200,sma50gtsma200co,sma50ltsma200,sma50ltsma200co,macdgtsignal,macdgtsignalco,macdltsignal,macdltsignalco,astral_buy,astral_sell,hammer,inverted_hammer,shooting_star,hanging_man,three_white_soldiers,three_black_crows,doji,three_line_strike,two_black_gapping,morning_star,evening_star,abandoned_baby,morning_doji_star,evening_doji_star
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-07 09:00:00,BTC-GBP,3600,12399.70,12524.87,12503.74,12437.04,10.807921,0.000000,1.000000,12437.040000,12437.040000,12437.040000,12437.040000,12437.040000,12437.040000,12437.040000,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.000000,0.000000,10.807921,0.00,12437.040000,87.830000,-37.340000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2018-01-07 10:00:00,BTC-GBP,3600,12403.08,12505.42,12437.04,12486.08,12.968296,0.003943,1.003943,12461.560000,12461.560000,12461.560000,12461.560000,12447.937778,12444.584615,12440.672593,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,3.912023,0.782405,23.776217,119.99,12444.045714,61.374286,-40.965714,False,True,True,True,False,False,True,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2018-01-07 11:00:00,BTC-GBP,3600,12430.00,12546.82,12486.08,12437.10,20.277987,-0.003923,1.000005,12453.406667,12453.406667,12453.406667,12453.406667,12445.529383,12443.433136,12440.407956,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,3.025180,1.230960,3.498230,-85.29,12443.053469,103.766531,-13.053469,True,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2018-01-07 12:00:00,BTC-GBP,3600,12261.00,12450.62,12430.00,12299.99,23.184213,-0.011024,0.988980,12415.052500,12415.052500,12415.052500,12415.052500,12413.187298,12421.364961,12430.006626,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,-8.641665,-0.743565,-19.685983,-662.74,12422.615831,28.004169,-161.615831,False,True,False,False,True,True,False,False,True,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2018-01-07 13:00:00,BTC-GBP,3600,12250.00,12357.14,12299.98,12310.09,15.635170,0.000821,0.989793,12394.060000,12394.060000,12394.060000,12394.060000,12390.276787,12404.245736,12421.123913,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,-16.878176,-3.970487,-4.050814,-79.42,12406.540712,-49.400712,-156.540712,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [43]:
df['next_close'] = df['close'].shift(-1) > df['close'] # 1 if next close is higher, else 0
df.loc[df['close'].shift(-1) < df['close'], 'next_close'] = -1
df['next_close'] = df['next_close'].astype(int) # 1 for up, 0 for hold, -1 for down
df = df[:-1]

df['next_close'].value_counts()

 1    15176
-1    14738
 0       66
Name: next_close, dtype: int64

In [44]:
df1 = df[[ 'close_cpc', 'next_close' ]]

In [45]:
df1.head()

,close_cpc,next_close
date,,
2018-01-07 09:00:00,1.000000,1
2018-01-07 10:00:00,1.003943,-1
2018-01-07 11:00:00,1.000005,-1
2018-01-07 12:00:00,0.988980,1
2018-01-07 13:00:00,0.989793,1


In [46]:
X = df1['close_cpc'] # input
y = df1['next_close'] # target

In [47]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [65]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22485,), (7495,), (22485,), (7495,))

In [66]:
# create the sequential model
network = Sequential()

In [67]:
activation = 'tanh'

# create the structure of the neural network
network.add(Dense(1, input_shape=(1,), activation=activation))
network.add(Dense(3, activation=activation))
network.add(Dense(3, activation=activation))
network.add(Dense(3, activation=activation))
network.add(Dense(1, activation=activation))

In [68]:
# compile the model
if activation == 'tanh':
    network.compile(optimizer='rmsprop', loss='hinge', metrics=['accuracy'])
elif activation == 'relu':
    #network.compile(optimizer='adam', loss='mse')
    network.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [69]:
# train the model
network.fit(X_train.values, y_train.values, epochs=100, batch_size=24, validation_data=(X_test.values, y_test.values), verbose=1, callbacks=[], shuffle=False)

Epoch 1/100
937/937 [==============================] - 3s 2ms/step - loss: 1.0016 - accuracy: 0.0024 - val_loss: 0.9965 - val_accuracy: 0.0017
Epoch 2/100
937/937 [==============================] - 1s 2ms/step - loss: 0.9972 - accuracy: 0.0024 - val_loss: 0.9902 - val_accuracy: 0.0017
Epoch 3/100
937/937 [==============================] - 1s 2ms/step - loss: 0.9933 - accuracy: 0.1749 - val_loss: 0.9842 - val_accuracy: 0.5121
Epoch 4/100
937/937 [==============================] - 1s 2ms/step - loss: 0.9895 - accuracy: 0.5071 - val_loss: 0.9795 - val_accuracy: 0.5121
Epoch 5/100
937/937 [==============================] - 1s 2ms/step - loss: 0.9866 - accuracy: 0.5071 - val_loss: 0.9768 - val_accuracy: 0.5121
Epoch 6/100
937/937 [==============================] - 1s 2ms/step - loss: 0.9850 - accuracy: 0.5071 - val_loss: 0.9755 - val_accuracy: 0.5121
Epoch 7/100
937/937 [==============================] - 1s 2ms/step - loss: 0.9842 - accuracy: 0.5071 - val_loss: 0.9748 - val_accuracy: 0.5121

In [71]:
# evaluate the predictions of the model
y_pred = network.predict(X_test.values)
y_pred = np.around(y_pred, 0)

In [72]:
# classification performance report
print(classification_report(y_test, y_pred))  

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      3644
           0       0.00      0.00      0.00        13
           1       0.51      1.00      0.68      3838

    accuracy                           0.51      7495
   macro avg       0.17      0.33      0.23      7495
weighted avg       0.26      0.51      0.35      7495



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Experiment #2**

* Multiple inputs
* Single output

In [70]:
df2 = df[[ 'volume', 'close_cpc', 'next_close' ]]

In [74]:
df2.head()

,volume,close_cpc,next_close
date,,,
2018-01-07 09:00:00,10.807921,1.000000,1
2018-01-07 10:00:00,12.968296,1.003943,-1
2018-01-07 11:00:00,20.277987,1.000005,-1
2018-01-07 12:00:00,23.184213,0.988980,1
2018-01-07 13:00:00,15.635170,0.989793,1


In [75]:
# initialise scaler
scaler = MinMaxScaler(feature_range=(0,1))

In [77]:
# scale our features
df_scaled = pd.DataFrame(scaler.fit_transform(df[[ 'volume', 'close_cpc', 'next_close' ]].values))
df_scaled.columns = [ 'volume', 'close_cpc', 'next_close' ]

In [78]:
df_scaled.head()

,volume,close_cpc,next_close
0,0.007059,0.223056,1.0
1,0.008474,0.224158,0.0
2,0.013262,0.223057,0.0
3,0.015166,0.219977,1.0
4,0.010221,0.220204,1.0


In [79]:
X = df_scaled[[ 'volume', 'close_cpc' ]]
y = df_scaled[[ 'next_close' ]]

In [80]:
X.shape, y.shape

((29980, 2), (29980, 1))

In [85]:
def create_window(data, window_size = 1):    
    data_s = data.copy()
    for i in range(window_size):
        data = pd.concat([data, data_s.shift(-(i + 1))], axis = 1)
        
    data.dropna(axis=0, inplace=True)
    return(data)

In [86]:
window = 4
df_window = create_window(X, window)

In [87]:
df_window.shape

(29976, 10)

In [88]:
df_window.head()

,volume,close_cpc,volume,close_cpc,volume,close_cpc,volume,close_cpc,volume,close_cpc
0,0.007059,0.223056,0.008474,0.224158,0.013262,0.223057,0.015166,0.219977,0.010221,0.220204
1,0.008474,0.224158,0.013262,0.223057,0.015166,0.219977,0.010221,0.220204,0.011563,0.222198
2,0.013262,0.223057,0.015166,0.219977,0.010221,0.220204,0.011563,0.222198,0.010830,0.220690
3,0.015166,0.219977,0.010221,0.220204,0.011563,0.222198,0.010830,0.220690,0.022441,0.213547
4,0.010221,0.220204,0.011563,0.222198,0.010830,0.220690,0.022441,0.213547,0.020416,0.210606


In [94]:
X_df_window = np.reshape(df_window.values, (df_window.shape[0], window+1, len(X.columns)))

In [110]:
print (X_df_window.shape)
print (df_window.iloc[:window,:])
print (X_df_window[0,:,:])

(29976, 5, 2)
     volume  close_cpc    volume  ...  close_cpc    volume  close_cpc
0  0.007059   0.223056  0.008474  ...   0.219977  0.010221   0.220204
1  0.008474   0.224158  0.013262  ...   0.220204  0.011563   0.222198
2  0.013262   0.223057  0.015166  ...   0.222198  0.010830   0.220690
3  0.015166   0.219977  0.010221  ...   0.220690  0.022441   0.213547

[4 rows x 10 columns]
[[0.00705945 0.22305596]
 [0.00847448 0.22415754]
 [0.01326226 0.22305731]
 [0.01516582 0.21997745]
 [0.01022126 0.22020432]]


In [96]:
y_df_window = np.array(df_scaled['next_close'][window:])

In [97]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [98]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22485, 2), (7495, 2), (22485, 1), (7495, 1))

In [99]:
def model_lstm (window, features):    
    # create the sequential model
    model = Sequential()

    # create the structure of the neural network
    model.add(LSTM(300, input_shape=(window, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(200, input_shape=(window, features), return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_initializer='uniform', activation='relu'))        
    model.add(Dense(1, kernel_initializer='uniform', activation='relu'))

    # compile the model
    model.compile(loss='mse', optimizer='adam')

    return model

In [106]:
# create model
model = model_lstm(window+1, len(X.columns))

In [111]:
# fit the model
model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test), verbose=1, callbacks=[], shuffle=False)

Epoch 1/100


ValueError: ignored

In [108]:
# evaluate the predictions of the model
y_pred = network.predict(X_test.values)
y_pred = np.around(y_pred, 0)

ValueError: ignored

In [109]:
# classification performance report
print(classification_report(y_test, y_pred))

ValueError: ignored